<a href="https://colab.research.google.com/github/Berkay86/DeepPerson/blob/main/POLARITY%20SENTIMENT%20ANALYSIS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Kütüphanelerimizi içeriye alalım.
#Veri İşlemleri
!pip3 install snscrape
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 4.0 MB/s 


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string


In [3]:
import re 
import textblob
from textblob import TextBlob

In [4]:
from wordcloud.wordcloud import WordCloud,STOPWORDS
!pip install emot
from emot.emo_unicode import UNICODE_EMOJI
lemmatizer=WordNetLemmatizer()
from wordcloud import ImageColorGenerator
from PIL import Image
import warnings
%matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 19 kB/s 
Using matplotlib backend: agg


In [5]:
query="(worldcup)"
tweets=[]
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
  if i>20000:
    break
  else:
        tweets.append([tweet.date,tweet.id,tweet.url,tweet.user.username,tweet.sourceLabel,tweet.user.location,tweet.content,tweet.likeCount,tweet.retweetCount])
df=pd.DataFrame(tweets,columns=["Date","ID","url","username","source","location","tweet","num_of_likes", "num_of_retweet"])
df.to_csv("sentiment.csv",encoding='utf-8')
df

,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,2022-12-20 20:33:11+00:00,1605300301011030017,https://twitter.com/Angelotralala/status/16053...,Angelotralala,Twitter Web App,,@_OmarAg @AsfandY32733432 @FrankKhalidUK Argen...,0,0
1,2022-12-20 20:33:11+00:00,1605300299270197248,https://twitter.com/Yemmie_/status/16053002992...,Yemmie_,Twitter for iPhone,Where the money resides 💰,Ronaldo and Piers didn’t think the timing of t...,0,0
2,2022-12-20 20:33:10+00:00,1605300296636190720,https://twitter.com/danieltyree/status/1605300...,danieltyree,Twitter Web App,"Sacramento, CA",@tariqpanja Hate to see it. He had an incredib...,0,0
3,2022-12-20 20:33:10+00:00,1605300295437029376,https://twitter.com/fIowerly/status/1605300295...,fIowerly,Twitter for Android,sara sheher 20,MY MAN is celebrating a world cup. yours is po...,0,0
4,2022-12-20 20:33:10+00:00,1605300295424409600,https://twitter.com/uw4ht/status/1605300295424...,uw4ht,Twitter for Android,,@EveeLaurynn Women are not the end result in t...,0,0
...,...,...,...,...,...,...,...,...,...
19996,2022-12-20 16:31:14+00:00,1605239410928369665,https://twitter.com/lucianoamorim94/status/160...,lucianoamorim94,Twitter Web App,"Litoral Paraibuco (PB/PE), BRA",#my365songs 2022 :: 11 DEC \n#345 :: #FIFAWorl...,0,0
19997,2022-12-20 16:31:13+00:00,1605239407665360897,https://twitter.com/jjkbless/status/1605239407...,jjkbless,Twitter for Android,she/her 23 ︎✹☽,@fifaworldcup_fr THANK YOU JUNGKOOK FOR GIVING...,7,2
19998,2022-12-20 16:31:12+00:00,1605239405681479680,https://twitter.com/db1anco/status/16052394056...,db1anco,Twitter for Android,In the North,Messi winning the World Cup doesn't makes him ...,0,0
19999,2022-12-20 16:31:12+00:00,1605239404351692806,https://twitter.com/lucianoamorim94/status/160...,lucianoamorim94,Twitter Web App,"Litoral Paraibuco (PB/PE), BRA",#my365songs 2022 :: 10 DEC \n#344 :: #FIFAWorl...,0,0


In [20]:
nltk.download('stopwords')
turk_stop_words=list(stopwords.words("turkish"))
emoji=list(UNICODE_EMOJI.keys())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
df["location"]=df["location"].fillna("Unknown")

In [22]:
def ProcessedTweets(text):
  text=text.lower()
  text=" ".join(re.sub("(@[A-Za-z0-9]+)|(^0-9A-Za-z \t]) |(\w+:\/\/\S+)"," ",text).split()) 
  text=re.sub(r'\@\w+|\#\w+|\d+', "", text)
  punct=str.maketrans("","",string.punctuation+string.digits)
  text=text.translate(punct)
  tokens=word_tokenize(text)
  filtered_words=[w for w in tokens if w not in turk_stop_words]
  filtered_words=[w for w in filtered_words if w not in emoji]
  lemmatizer=WordNetLemmatizer()
  lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
  text=" ".join(lemma_words)
  return text



In [23]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
df["Processed_Tweets"]=df["tweet"].apply(ProcessedTweets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
df["Processed_Tweets"]

0        argentina had world cup before messi came alon...
1        ronaldo and pier didn ’ t think the timing of ...
2        hate to see it he had an incredible world cup ...
3        my man is celebrating a world cup yours is pos...
4        woman are not the end result in this world cup...
                               ...                        
19996    dec giorgio moroder project to be number one s...
19997    fr thank you jungkook for giving u the best wo...
19998    messi winning the world cup doesnt make him th...
19999    dec jason derulo color cocacola anthem for the...
20000    are qatar ’ s world cup stadium the future of ...
Name: Processed_Tweets, Length: 20001, dtype: object

In [25]:
def polarity(tweet):
  return TextBlob(tweet).sentiment.polarity
def sentimenttextblob(polarity):
  if polarity<0:
    return "Negative"
  elif polarity>0:
    return "Positive"


In [12]:
df["Polarity"]=df["Processed_Tweets"].apply(polarity)
df["Sentiment"]=df["Polarity"].apply(sentimenttextblob)
sent=df["Sentiment"].value_counts()
sent
df.to_csv("sentiment.csv",encoding='utf-8')